In [1]:
#Rentrer votre chemin vers le git
import os

os.chdir('C:\\Users\\theo.roudil-valentin\\Documents\\Codes')

# Notebook pour le clustering non-supervisé des titres
Ce code est divisé en deux parties :
## 1. Ajout, affinage des features
## 2. Algorithme(s) de clustering

In [2]:
import pandas as pd, numpy as np
import time
import os

try:
    # Si vous avez accès à K dans le fichier setup.py
    os.chdir('2021-NLP_AE\Data\Bagging_model')
    base_phrase_clean=pd.read_csv('base_phrase_clean.csv',sep=None)
except:
    # Sinon, vous avez écrit la base dans le dossier présent
    os.chdir('2021-NLP_AE\Pipeline\Extraction_sommaire')
    base_phrase_clean=pd.read_csv('base_phrase_clean.csv',sep=None)

print('Il y a ',len(np.unique(base_phrase_clean.num_etude)),"études")
base_phrase_clean

c:\Users\theo.roudil-valentin\venv\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  


Il y a  148 études


,phrase,num_etude,0,1,2,3,4,5,6,7,...,31,32,33,34,35,36,37,38,39,40
0,<title>EI Munchhouse_V3</title>,1003691,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"<h1><a name=""bookmark0""></a><span class=""font2...",1003691,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"<p><span class=""font20"" style=""font-weight:bol...",1003691,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"<p><span class=""font14"">Agence Aquitaine</span...",1003691,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"<p><span class=""font14"">SAFEGE</span></p>",1003691,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2443029,"<span class=""font21"">Disponible sur le site de...",238301,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2443030,"<a href=""http://www.clefdeschamps.info/""><span...",238301,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2443031,"<p><sup><a href=""#footnote14"">14</a></sup><a n...",238301,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2443032,"<span class=""font0"" style=""font-weight:bold;"">...",238301,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 1. Ajout et affinage des features

#### On rajoute les features :
* nombre de mots,
* nombre de caractères
* mais également nombre de caractères de la ligne html

In [7]:
from bs4 import BeautifulSoup
base_phrase_clean['phrase_2']=[BeautifulSoup(i,"html.parser").text for i in base_phrase_clean.phrase] # On nettoie le html présent dans le texte
base_phrase_clean['f_mots']=[len(i.split(' ')) for i in base_phrase_clean.phrase_2] #On compte le nombre de mots
base_phrase_clean['f_carac']=[len(i) for i in base_phrase_clean.phrase_2] # On compte le nombre de caractères
base_phrase_clean['html_carac']=[len(i) for i in base_phrase_clean.phrase] # On compte le nombre de caractères html
base_phrase_clean.shape

(2443034, 47)

#### On rajoute indicatrice sommaire

In [4]:
import re
from unidecode import unidecode

def sommaire(s):
    '''
    Fonction cherchant l'indice du début du sommaire dans la liste de phrase.
    '''
    somm=['SOMMAIRE','sommaire','Sommaire','ommaire']
    s=list(s)
    indice=[s.index(k) for k in somm if k in s]
    if len(indice)==0:
        indice=[i for i in range(len(s)) if np.sum([1 if som in s[i] else 0 for som in somm])>0]
        if len(indice)>2:
            indice=[indice[0]]
        if len(indice)==0:
            from unidecode import unidecode
            technique=[unidecode('table des matières'.lower()),'non technique']
            indice=[i for i in range(len(s)) if np.sum([1 if som in unidecode(s[i].lower()) else 0 for som in technique])>0]
    #[list(verif_theo[verif_theo.label_k==0][verif_theo.num_etude==verif_theo.num_etude[0]].phrase_2.values).index(k) for k in somm if k in list(verif_theo[verif_theo.label_k==0][verif_theo.num_etude==verif_theo.num_etude[0]].phrase_2.values)]
    return indice

def light_cleaning(s):
    '''
    Fonction qui produit un nettoyage sommaire d'un string s
    @s : un texte de type string
    '''
    s=unidecode(s)
    s=re.sub(r'[^A-Za-z]',' ',s)
    s=s.lower()
    s=s.split(' ')
    s=list(np.unique(s))
    try:
        s.remove('')
        return s
    except:
        return s


def indice_relou(liste,indice_sommaire,seuil,seuil_c):
    '''
    Fonction qui permet de produire l'indicatrice sommaire pour les études récalcitrantes : c'est-à-dire dont le sommaire est difficilement détectable.
    Par exemple à cause de mauvaises répétitions des titres, d'absence de mots indiquants le début ou la fin du sommaire etc...
    '''
    output=None
    x=0
    #if procede=='basic':
    while output is None:
        x+=1
        a=light_cleaning(liste[indice_sommaire[0]+x]) 
        for j in range(indice_sommaire[0]+1+1,len(liste)):
            b=light_cleaning(liste[j])
            cnt = 0
            if len(a)==1:
                s=0
                for i in a:
                    if type(b)==str:
                        if re.search(i, b) is not None:
                            cnt = cnt + 1
                cnt=cnt/len(a)
                count=len(set(a)&set(b))/len(a)
            elif len(a)>1:
                aa=''.join([i for i in a])
                for i in aa:
                    if type(b)==str:
                        if re.search(i, b) is not None:
                            cnt = cnt + 1
                cnt=cnt/len(aa)
                count=len(set(aa)&set(b))/len(aa)
                s=0
                if type(b)==str:
                    for i in b:
                        if i in a:
                            s=1

            if (b in a) or (a in b) or (cnt>seuil) or (count>seuil_c) or (s>0):
                output=j
                #vector=np.ones(len([indice_sommaire[0]:j]))
                break
            else:
                continue
  #  elif procede=='further':
   #     a=light_cleaning(liste[indice_sommaire[0]]) 
    #else:
       # raise ValueError("Hum... Êtes-vous sûr que vous avez correctement spécifié le variable procede ? \nLes valeurs possibles sont 'basic' et 'further' (pour les études récalcitrantes).")
        
    return output

def ind_titres(verif,brutal=None,seuil_indice=100,seuil_sommaire=50,seuil=0.35,seuil_c=0.35):
    '''
    Fonction générale produisant l'indicatrice du sommaire, basée sur de nombreuses règles. Cette fonction s'est enrichie de l'analyse des soucis que nous voyions en
    aval du processus. Nous avons donc fait des allers retours pour l'améliorer et intégrer des règles informatiques.
    @verif : base de données des lignes html découpées, avec comme variables celles produites par le code 1 du dossier.
    @brutal : None ou type(int), si type==int ont pas prendre les brutal premières lignes après le mot sommaire (permet de toujours produire la variable mais peut manquer de précision). Cette option a été mise en place quand on ne réussissait pas à produire l'indicatrice.
    @seui_indice :  sanity check, seuil pour l'indice d'apparition du mot sommaire. Si l'indice dépasse ce seuil, méfiance. La fonction passe alors à brutal.
    @seuil_sommaire : taille minimum pour un sommaire, en dessous, on continuera de chercher car le sommaire détecté nous semblera trop petit
    @seuil et seuil_c : seuil pour détecter les sommaires des études récalcitrantes, voir fonction précédente.
    '''
    verif_theo=verif.copy()
    titres={}
    vector_=[]
    indexes = np.unique(verif_theo.num_etude, return_index=True)[1]
    #print(indexes)
    try:
        interval=[verif_theo.num_etude[index] for index in sorted(indexes)]
    except:
        verif_theo=verif_theo.reset_index(drop=True)
        interval=[verif_theo.num_etude[index] for index in sorted(indexes)]
    #print(interval)
    h=0
    for i in interval:
        print("On étudie l'étude :",i)
        liste=verif_theo[verif_theo.num_etude==i].phrase_2.values
        vector=np.zeros(len(liste))
        vector_.append(vector)
        h=h+len(vector)
        #vector_.append(vector)
        indice_sommaire=sommaire(liste)
        if len(indice_sommaire)>0:
            a=light_cleaning(liste[indice_sommaire[0]+1]) #hypothèse donc que le 1er titre vient après le sommaire
            print("L'indice de début de sommaire pour l'étude est",indice_sommaire[0],a)
            for j in range(indice_sommaire[0]+1+1,len(liste)):
                b=light_cleaning(liste[j])
                if a==b:
                    vector_[interval.index(i)][indice_sommaire[0]:j]=1
                    break
                else:
                    continue
            ss=np.sum(vector_[interval.index(i)])
            if ss>0:
                if ss>seuil_sommaire:
                    print("Il y a un sommaire pour l'étude.")
                else:
                    try:
                        vector_[interval.index(i)][indice_sommaire[0]:brutal+indice_sommaire[0]]=np.ones(len(range(indice_sommaire[0],brutal+indice_sommaire[0])))
                        print("On a pris les lignes de ",indice_sommaire[0],"à",indice_sommaire[0]+brutal)
                    except:
                        vector_[interval.index(i)][indice_sommaire[0]:brutal]=np.ones(len(range(indice_sommaire[0],brutal)))
                        print("On a pris les lignes de ",indice_sommaire[0],"à",brutal)
            elif np.sum(vector_[interval.index(i)])==0:
                print('On tente la nouvelle méthode.')
                j=indice_relou(liste,indice_sommaire,seuil,seuil_c)
                if j is not None:
                    vector_[interval.index(i)][indice_sommaire[0]:j]=np.ones(len(range(indice_sommaire[0],j)))
                    ss=np.sum(vector_[interval.index(i)])
                    if ss>seuil_sommaire:
                        print("Il y a un sommaire pour l'étude.")
                    elif (ss>0) and (ss<seuil_sommaire) and (brutal is not None) and (indice_sommaire[0]<seuil_indice):
                        print("Hum... Le sommaire semble petit :",ss)
                        try:
                            vector_[interval.index(i)][indice_sommaire[0]:brutal+indice_sommaire[0]]=np.ones(len(range(indice_sommaire[0],brutal+indice_sommaire[0])))
                            print("On a pris les lignes de ",indice_sommaire[0],"à",indice_sommaire[0]+brutal)
                        except:
                            vector_[interval.index(i)][indice_sommaire[0]:brutal]=np.ones(len(range(indice_sommaire[0],brutal)))
                            print("On a pris les lignes de ",indice_sommaire[0],"à",brutal)
                        
                    else:
                        print("Pas de sommaire pour l'étude.")
                        try:
                            vector_[interval.index(i)][indice_sommaire[0]:brutal+indice_sommaire[0]]=np.ones(len(range(indice_sommaire[0],brutal+indice_sommaire[0])))
                            print("On a pris les lignes de ",indice_sommaire[0],"à",indice_sommaire[0]+brutal)
                        except:
                            vector_[interval.index(i)][indice_sommaire[0]:brutal]=np.ones(len(range(indice_sommaire[0],brutal)))
                            print("On a pris les lignes de ",indice_sommaire[0],"à",brutal)
                else:
                    print("Pour l'étude",i,"on n'a pas trouvé d'indice pour débuter le sommaire.")
        else:
            print("Pas de sommaire pour l'étude.")
    return vector_

Par exemple le problème de l'étude 101872 c'est que lors de la création de variable, le premier élément après le sommaire n'est pas un titre. La variable d'indicatrice sommaire est donc mal codée...

In [ ]:
for i in base_phrase_clean[base_phrase_clean.num_etude==101872].phrase[:100]:
    print('\n',i,'\n')

In [8]:
start=time.time()
#3ème version, ici on prend brutalement les 200 lignes après le mot "sommaire"
ouais=ind_titres(base_phrase_clean,brutal=400)
end=time.time()
print("L'indicatrice a pris",round((end-start)/60),"minutes.")
import functools
import operator
c=functools.reduce(
    operator.iconcat,ouais,[])
#print(len(c))
base_phrase_clean['ind_sommaire']=c

On étudie l'étude : 1003691
L'indice de début de sommaire pour l'étude est 44 ['c', 'non', 'ra', 'suma', 'technique']
On tente la nouvelle méthode.
Il y a un sommaire pour l'étude.
On étudie l'étude : 100689
L'indice de début de sommaire pour l'étude est 101 ['demandeur', 'du', 'presentation']
Il y a un sommaire pour l'étude.
On étudie l'étude : 100707
L'indice de début de sommaire pour l'étude est 87 ['d', 'le', 'maitre', 'ouvrage']
On tente la nouvelle méthode.
Il y a un sommaire pour l'étude.
On étudie l'étude : 1007353
L'indice de début de sommaire pour l'étude est 53 ['non', 'resume', 'technique']
Il y a un sommaire pour l'étude.
On étudie l'étude : 1007765
L'indice de début de sommaire pour l'étude est 0 ['ambule', 'c', 'contexte', 'le', 'pra']
Il y a un sommaire pour l'étude.
On étudie l'étude : 1008101
Pas de sommaire pour l'étude.
On étudie l'étude : 1012199
L'indice de début de sommaire pour l'étude est 235 []
On a pris les lignes de  235 à 635
On étudie l'étude : 1017145
L'i

In [10]:
numero=np.unique(base_phrase_clean.num_etude)
num_relou=[i for i in numero if np.sum(base_phrase_clean[base_phrase_clean.num_etude==i].ind_sommaire)==0]
index=[True if base_phrase_clean.num_etude[i] in num_relou else False for i in base_phrase_clean.index]
base_relou=base_phrase_clean[index]
print("Les documents vides représentent :",round(len(base_relou)/len(base_phrase_clean)*100,2),'% des lignes de la base initiale')
print("On a réussi à traiter ",round((1-len(np.unique(base_relou.num_etude))/len(numero))*100,2),"% des documents.")
print("Il y a ",len(np.unique(base_relou.num_etude)),"études vides")

Les documents vides représentent : 1.0 % des lignes de la base initiale
On a réussi à traiter  96.62 % des documents.
Il y a  5 études vides


In [12]:
base_phrase_clean[base_phrase_clean.ind_sommaire==1].phrase_2

44                                                  Sommaire
45                                \n1.RÃ©sumÃ© non technique
46               \n1.1 PrÃ©sentation synthÃ©tique du projet 
47                     \n1.2 SynthÃ¨se de lâ€™Ã©tat initial 
48         \n1.3 SynthÃ¨se des impacts et des mesures ass...
                                 ...                        
2435134                                   TonalitÃ© marquÃ©e
2435135    Lâ€™Ã©tude dâ€™impact sonore a Ã©tÃ© rÃ©alisÃ©...
2435136          VI.4. METHODOLOGIE DE Lâ€™ETUDE ACOUSTIQUE.
2435137                          \n11.3.7.1. RÃ©glementation
2435138    Lâ€™ArrÃªtÃ© du 26 aoÃ»t 2011 relatif aux inst...
Name: phrase_2, Length: 176992, dtype: object

Après avoir analysé les résultats en bout de pipeline : on a intérêt à bien travailler cette variable, beaucoup de gains possibles à travers elle.

On va donc raffiner cette variable.

De plus, en sortant les phrases liées à l'indicatrice de sommaire, on voit que celle-ci ne peut pas être utilisée comme règle directe, car il y a beaucoup de déchets, plus qu'après tout le pipe (bien qu'il soit assez conséquent, bien que peu couteux en termes de calcul)

In [ ]:
titres=base_phrase_clean[base_phrase_clean.ind_sommaire==1].phrase_2
for i in titres.index:
    print(titres[i])

In [14]:
def raffinement_sommaire(base,a,b):
    c=[1 if len(i)>a or len(i)<b else 0 for i in base.phrase_2]#np.zeros(len(base))
    c_=np.zeros(len(base))
    for i in range(len(base)):
        try:
            k=int(base.phrase_2[i])
            #print(k)
            c_[i]=1
        except:
            continue
    base['sommaire_longueur']=c
    base['sommaire_int']=c_
    return base

In [15]:
start=time.time()
base_=raffinement_sommaire(base_phrase_clean,180,3)
base_.sommaire_int=base_.sommaire_int*base_.ind_sommaire
base_.sommaire_longueur=base_.sommaire_longueur*base_.ind_sommaire
base_
end=time.time()
print("Durée",round((end-start)/60),"minutes.")

Durée 0 minutes.


In [ ]:
for i in base_[base_.sommaire_int>0].phrase_2.index:
    print(titres[i])

In [ ]:
for i in base_[base_.sommaire_longueur==1].phrase_2.index:
    print(titres[i])

### Gestion des caractères spéciaux

In [250]:
def caractere_spec(base,weird=["□","■"]):
    base['caractere_spec']=[1 if np.mean([int(weird[i] in base.phrase_2[K]) for i in range(len(weird))])>0 else 0
 for K in range(len(base.phrase_2))]
    return base
start=time.time()
base=caractere_spec(base_)
end=time.time()
print("Durée :",round((end-start)/60),"minutes.")
base[base.caractere_spec==1]

Durée : 8 minutes.


,phrase,num_etude,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,phrase_2,f_mots,f_carac,html_carac,ind_sommaire,sommaire_longueur,sommaire_int,caractere_spec
1138,"<span class=""font34"" style=""font-weight:bold;""...",1003691,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,□,1,1,55,0.0,0.0,0.0,1
1141,"<span class=""font34"" style=""font-weight:bold;""...",1003691,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,□,1,1,55,0.0,0.0,0.0,1
1144,"<span class=""font34"" style=""font-weight:bold;""...",1003691,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,□,1,1,55,0.0,0.0,0.0,1
1321,"<td colspan=""4"" style=""vertical-align:bottom;""...",1003691,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,\n■ ■■,2,5,92,0.0,0.0,0.0,1
1357,"<span class=""font34"" style=""font-weight:bold;""...",1003691,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,□,1,1,55,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2442371,"<span class=""font5""><a name=""bookmark1127""></a...",238301,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,■^,2,3,57,0.0,0.0,0.0,1
2442469,"<span class=""font5"">■</span>",238301,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,■,1,1,28,0.0,0.0,0.0,1
2442482,"<span class=""font5"" style=""font-style:italic;""...",238301,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,■,1,1,55,0.0,0.0,0.0,1
2442604,"<span class=""font5"">■</span>",238301,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,■,1,1,28,0.0,0.0,0.0,1


In [14]:
print("Il y a",round((len(base[base.caractere_spec==1])/len(base))*100,2),"% de lignes dans la base avec des caractères spéciaux.")

Il y a 1.11 % de lignes dans la base avec des caractères spéciaux.


### On a notre matrice d'entraînement
attention à bien gérer les indices au niveau des colonnes pour prendre les bonnes !

In [251]:
train=base_phrase_clean.iloc[:,2:]
train=pd.concat([train.iloc[:,:-8],train.iloc[:,-7:]],axis=1)
train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,f_mots,f_carac,html_carac,ind_sommaire,sommaire_longueur,sommaire_int,caractere_spec
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2,16,31,0.0,0.0,0.0,0
1,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2,113,0.0,0.0,0.0,0
2,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,4,65,0.0,0.0,0.0,0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,16,51,0.0,0.0,0.0,0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,41,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2443029,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,68,96,0.0,0.0,0.0,0
2443030,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,30,103,0.0,0.0,0.0,0
2443031,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2,73,0.0,0.0,0.0,0
2443032,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,65,0.0,0.0,0.0,0


### On va maintenant faire une PCA sur la partie creuse de la matrice puis standardiser

#### Ensuite une PCA sur le ventre creux de la matrice

In [20]:
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.model_selection import train_test_split
pca=PCA(3)
p=41
#pca.fit(train_.iloc[:,1:p])
#print(pca.explained_variance_ratio_)

In [ ]:
print("Variance expliquée :",round(pca.explained_variance_ratio_.sum()*100,2),"%")

In [ ]:
ordo_p={}
P=range(31,42)
for p in P:
    ordo=[]
    for i in range(1,15):
        pca=PCA(i)
        pca.fit(train_.iloc[:,1:p])
        ordo.append(pca.explained_variance_ratio_.sum())
    ordo_p[p]=ordo

In [ ]:
for i in P:
    print(len(ordo_p[i]))

In [ ]:
import matplotlib.pyplot as plt
f,a=plt.subplots(1,figsize=(14,8))
absi=range(14)
for i in P:
    a.plot(absi,ordo_p[i])
a.set(xlabel="Nombre de variables",ylabel='Variance expliquée',
      title="Variance expliquée en fonction du nombre d'axes de la PCA")
plt.legend([p for p in P])

On choisit donc p=33 et i=14

In [252]:
pca=PCA(12)
ventre=pca.fit_transform(train_.iloc[:,1:33])
print(ventre.shape)
ventre

(2443034, 12)


array([[-7.57411820e+01, -3.61537183e+01,  9.47331037e-01, ...,
         5.12262957e-01,  5.89025348e-02, -1.42566174e-01],
       [-2.79741416e+01,  3.19482314e+01, -6.69773035e-01, ...,
         3.35668010e-01, -1.64927465e-02,  2.16633987e-02],
       [-6.03491675e+01, -3.53404430e+00, -7.91909513e-02, ...,
        -3.93859361e-02, -2.47152902e-01,  1.66270262e-01],
       ...,
       [-5.61227855e+01,  3.52124763e+00, -4.44584103e-01, ...,
        -3.87409779e-02,  2.90545073e-01, -6.68714497e-01],
       [-6.23488612e+01, -1.56947937e+00, -1.50834875e+00, ...,
         1.02019227e-01,  8.38998717e-02, -1.19895989e-01],
       [ 1.27859266e+02, -2.66681626e+01,  4.80989981e+00, ...,
         4.75217856e-01,  2.57648651e-02, -2.38361846e-02]])

In [259]:
ventre=pd.DataFrame(ventre,columns=['ventre_'+str(i) for i in range(ventre.shape[1])])
train__=pd.concat([train_.iloc[:,0],pd.DataFrame(ventre),train.iloc[:,33:]],axis=1)
train__

,0,ventre_0,ventre_1,ventre_2,ventre_3,ventre_4,ventre_5,ventre_6,ventre_7,ventre_8,ventre_9,ventre_10,ventre_11,33,34,35,36,37,38,39,40,f_mots,f_carac,html_carac,ind_sommaire,sommaire_longueur,sommaire_int,caractere_spec
0,0.0,-75.741182,-36.153718,0.947331,0.524724,0.774977,0.185759,-0.319236,0.455361,-0.065684,0.512263,0.058903,-0.142566,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2,16,31,0.0,0.0,0.0,0
1,44.0,-27.974142,31.948231,-0.669773,1.024560,0.508571,-0.525606,-0.551758,0.265110,-0.041230,0.335668,-0.016493,0.021663,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2,113,0.0,0.0,0.0,0
2,20.0,-60.349167,-3.534044,-0.079191,0.203887,0.470977,-0.656074,-0.574462,-0.481015,-0.118332,-0.039386,-0.247153,0.166270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,4,65,0.0,0.0,0.0,0
3,1.0,-61.665856,-21.942566,0.926980,0.396274,-0.085157,0.430824,0.293239,-0.446495,-0.122702,0.055701,-0.205235,0.139373,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,16,51,0.0,0.0,0.0,0
4,1.0,-75.833499,-21.988102,0.402921,0.397976,-0.104315,0.441795,0.309084,-0.463824,-0.125930,0.046478,-0.210323,0.141424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,41,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2443029,1.0,7.595472,-27.226777,-1.069930,0.682739,-0.249227,0.197026,0.220505,0.289631,-0.018945,0.267959,0.068317,-0.098457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,68,96,0.0,0.0,0.0,0
2443030,1.0,-15.347110,5.383671,3.801120,1.269050,0.274064,0.614862,0.134279,0.189666,-0.093822,-0.003617,0.498173,-0.729496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,30,103,0.0,0.0,0.0,0
2443031,0.0,-56.122786,3.521248,-0.444584,0.562114,1.023024,0.806830,-0.131741,-0.378850,-0.230992,-0.038741,0.290545,-0.668714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2,73,0.0,0.0,0.0,0
2443032,1.0,-62.348861,-1.569479,-1.508349,0.891659,-0.449609,-0.748661,-0.188959,0.108123,-0.027538,0.102019,0.083900,-0.119896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,65,0.0,0.0,0.0,0


In [255]:
len(np.unique(train.iloc[:,0].values))

def scaling(k,k_min,k_max):
    return (k-k_min)/(k_max-k_min)

def scaler(X,z=10):
    if len(np.unique(X.values)>2):
        from joblib import Parallel,delayed
        from functools import partial
        X_min=min(X)
        X_max=max(X)
        scal=partial(scaling,k_min=X_min,k_max=X_max)
        X_=Parallel(n_jobs=z)(delayed(scal)(X[i]) for i in range(len(X)))
        return X_
    else:
        return X

dico_scale={}
train_=train__.copy()
from joblib import Parallel,delayed
from functools import partial
import psutil
cpu=psutil.cpu_count()

start=time.time()
#for i in range(train_.shape[1]):
 #   if len(np.unique(train.iloc[:,i].values))>2:
 #       dico_scale[i]=scaler(train_.iloc[:,i])
  #      train_.iloc[:,i]=dico_scale[i]
    #else:
     #   continue
scal=partial(scaler,z=cpu)
c=Parallel(n_jobs=cpu)(delayed(scal)(train_.iloc[:,i]) for i in range(train_.shape[1]))
end=time.time()

print("Durée :",round((end-start)/60,2),"minutes")

#dico_scale.keys()

#train_.iloc[:,1:41]

Durée : 32.25 minutes


In [263]:
tt=pd.DataFrame(c).T
train_=pd.concat([train.iloc[:,0],tt,train.iloc[:,33:]],axis=1)
train_=pd.concat([train_.iloc[:,:21],train_.iloc[:,22:]],axis=1)
train_.columns=train__.columns
train_

### On vire les colonnes redondantes
A priori aucune après ce nouveau processing

In [ ]:
def pandas_col_linear_dep(A):
    corr=[]
    for i in range(A.shape[1]):
        for j in range(A.shape[1]):
            a=np.corrcoef(A.iloc[:,i],A.iloc[:,j])[1][0]
            if (a==1) and (i!=j):
                corr.append([i,j])
    #a=[(np.corrcoef(A.iloc[:,i],A.iloc[:,j])[1][0],i,j) for i in A.shape[1] for j in A.shape[1] if
        #(i!=j) and (np.corrcoef(A.iloc[:,i],A.iloc[:,j])[1][0]==1)]
    return corr

start=time.time()
corr=pandas_col_linear_dep(train__)
end=time.time()
print('Durée :',round((end-start)/60,2),' minutes')

corr

In [ ]:
if len(corr)>0:
    for i in corr[0:int(len(corr)/2)]:
        try:
            train__=train__.drop([str(i[1])],axis=1)
        except:
            continue

In [ ]:
train__=tt

## 2. Clustering non-supervisé
On peut faire varier le nombre de cluster, mais pour le moment, je l'ai fixé à 2 : conceptuellement les titres et le reste. Mais en termes machine cela peut être différent, on souhaite que la machine le perçoive comme nous. L'ajout d'un cluster peut être intéressant et aider à travailler sur les phrases qui se rapprochent des titres à cause des problèmes d'OCRisation ou autres par exemple

In [271]:
n_cluster=2

In [273]:
from sklearn.cluster import KMeans
kmeansmodel=KMeans(n_clusters=n_cluster,n_init=100,max_iter=500)
start=time.time()
kmeans=kmeansmodel.fit(train_)
end=time.time()
print('Durée :',round((end-start)/60,2),' minutes')
train_['label_k']=kmeans.labels_

Durée : 4.16  minutes


In [274]:
import pickle
pickle.dump(kmeans,open("/kmeans_model_new.pickle",'wb'))

In [26]:
import pickle

kmeans_model=pickle.load(open("/kmeans_model_new.pickle",'rb'))
train__['label_k']=kmeans.labels_

Ne pas faire tourner les clusterings suivants

In [ ]:
from sklearn.cluster import SpectralClustering
sc = SpectralClustering(n_cluster,  n_init=100,
                            assign_labels='discretize')
start=time.time()
sc.fit(train.iloc[:,:-1])
end=time.time()
print('Durée :',round((end-start)/60,2),' minutes')
train['label_sc']=sc.labels_

In [ ]:
from sklearn.cluster import DBSCAN
start=time.time()
clustering = DBSCAN(eps=0.1, min_samples=100).fit(train.iloc[:,:-1])
end=time.time()
print('Durée :',round((end-start)/60,2),' minutes')
train['label_dbs']=clustering.labels_

On réorganise la base et on la sauvegarde

In [275]:
train__

,0,ventre_0,ventre_1,ventre_2,ventre_3,ventre_4,ventre_5,ventre_6,ventre_7,ventre_8,ventre_9,ventre_10,ventre_11,33,34,35,36,37,38,39,40,f_mots,f_carac,html_carac,ind_sommaire,sommaire_longueur,sommaire_int,caractere_spec
0,0.0,-75.741182,-36.153718,0.947331,0.524724,0.774977,0.185759,-0.319236,0.455361,-0.065684,0.512263,0.058903,-0.142566,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2,16,31,0.0,0.0,0.0,0
1,44.0,-27.974142,31.948231,-0.669773,1.024560,0.508571,-0.525606,-0.551758,0.265110,-0.041230,0.335668,-0.016493,0.021663,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2,113,0.0,0.0,0.0,0
2,20.0,-60.349167,-3.534044,-0.079191,0.203887,0.470977,-0.656074,-0.574462,-0.481015,-0.118332,-0.039386,-0.247153,0.166270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,4,65,0.0,0.0,0.0,0
3,1.0,-61.665856,-21.942566,0.926980,0.396274,-0.085157,0.430824,0.293239,-0.446495,-0.122702,0.055701,-0.205235,0.139373,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,16,51,0.0,0.0,0.0,0
4,1.0,-75.833499,-21.988102,0.402921,0.397976,-0.104315,0.441795,0.309084,-0.463824,-0.125930,0.046478,-0.210323,0.141424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,41,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2443029,1.0,7.595472,-27.226777,-1.069930,0.682739,-0.249227,0.197026,0.220505,0.289631,-0.018945,0.267959,0.068317,-0.098457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,68,96,0.0,0.0,0.0,0
2443030,1.0,-15.347110,5.383671,3.801120,1.269050,0.274064,0.614862,0.134279,0.189666,-0.093822,-0.003617,0.498173,-0.729496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,30,103,0.0,0.0,0.0,0
2443031,0.0,-56.122786,3.521248,-0.444584,0.562114,1.023024,0.806830,-0.131741,-0.378850,-0.230992,-0.038741,0.290545,-0.668714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2,73,0.0,0.0,0.0,0
2443032,1.0,-62.348861,-1.569479,-1.508349,0.891659,-0.449609,-0.748661,-0.188959,0.108123,-0.027538,0.102019,0.083900,-0.119896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,65,0.0,0.0,0.0,0


In [276]:
#train__.columns=["0"]+['ventre_'+str(i) for i in range(ventre.shape[1])]+list(train__.columns[15:].values)
df=pd.concat([base.iloc[:,:2],base.loc[:,'phrase_2'],train__],axis=1)
df

,phrase,num_etude,phrase_2,0,ventre_0,ventre_1,ventre_2,ventre_3,ventre_4,ventre_5,ventre_6,ventre_7,ventre_8,ventre_9,ventre_10,ventre_11,33,34,35,36,37,38,39,40,f_mots,f_carac,html_carac,ind_sommaire,sommaire_longueur,sommaire_int,caractere_spec
0,<title>EI Munchhouse_V3</title>,1003691,EI Munchhouse_V3,0.0,-75.741182,-36.153718,0.947331,0.524724,0.774977,0.185759,-0.319236,0.455361,-0.065684,0.512263,0.058903,-0.142566,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2,16,31,0.0,0.0,0.0,0
1,"<h1><a name=""bookmark0""></a><span class=""font2...",1003691,06,44.0,-27.974142,31.948231,-0.669773,1.024560,0.508571,-0.525606,-0.551758,0.265110,-0.041230,0.335668,-0.016493,0.021663,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2,113,0.0,0.0,0.0,0
2,"<p><span class=""font20"" style=""font-weight:bol...",1003691,2019,20.0,-60.349167,-3.534044,-0.079191,0.203887,0.470977,-0.656074,-0.574462,-0.481015,-0.118332,-0.039386,-0.247153,0.166270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,4,65,0.0,0.0,0.0,0
3,"<p><span class=""font14"">Agence Aquitaine</span...",1003691,Agence Aquitaine,1.0,-61.665856,-21.942566,0.926980,0.396274,-0.085157,0.430824,0.293239,-0.446495,-0.122702,0.055701,-0.205235,0.139373,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,16,51,0.0,0.0,0.0,0
4,"<p><span class=""font14"">SAFEGE</span></p>",1003691,SAFEGE,1.0,-75.833499,-21.988102,0.402921,0.397976,-0.104315,0.441795,0.309084,-0.463824,-0.125930,0.046478,-0.210323,0.141424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,41,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2443029,"<span class=""font21"">Disponible sur le site de...",238301,Disponible sur le site de Réseau de Transport ...,1.0,7.595472,-27.226777,-1.069930,0.682739,-0.249227,0.197026,0.220505,0.289631,-0.018945,0.267959,0.068317,-0.098457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,68,96,0.0,0.0,0.0,0
2443030,"<a href=""http://www.clefdeschamps.info/""><span...",238301,http://www.clefdeschamps.info/,1.0,-15.347110,5.383671,3.801120,1.269050,0.274064,0.614862,0.134279,0.189666,-0.093822,-0.003617,0.498173,-0.729496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,30,103,0.0,0.0,0.0,0
2443031,"<p><sup><a href=""#footnote14"">14</a></sup><a n...",238301,14,0.0,-56.122786,3.521248,-0.444584,0.562114,1.023024,0.806830,-0.131741,-0.378850,-0.230992,-0.038741,0.290545,-0.668714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2,73,0.0,0.0,0.0,0
2443032,"<span class=""font0"" style=""font-weight:bold;"">...",238301,,1.0,-62.348861,-1.569479,-1.508349,0.891659,-0.449609,-0.748661,-0.188959,0.108123,-0.027538,0.102019,0.083900,-0.119896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,65,0.0,0.0,0.0,0


In [277]:
pickle.dump(df,open("/base_pour_Bagging_final_new.pickle",'wb'))